In [1]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [2]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator
import model_definitions

### Open input file

In [18]:
SEQUENCE_LENGTH=20
ONE_CAR=True
PATH=r"D:\Users\Students\Daniele\records\OneCar_db.txt"
BATCH_SIZE=256*2
FEATURES=7 
FEATURES_PREDICTED=3

In [19]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [29]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
df=Generator.single_care_dataframe(PATH)

In [30]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

dfs=[]
for i in range(len(df)):
    dfs.append(SubtractDF(df[i]))
    #dfs.append(df[i])

united=dfs[0]
for i in range(1,len(df)):
    united=pd.concat([united,dfs[i] ],ignore_index=True)

In [31]:
united.tail()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
5310813,1.1339,1.2732,-1.32906,0.32719,-2.35638,0.000261,0.440120,0.078774,8,16,0.399682,0.073825,0.06,167,2
5310814,1.0530,1.2877,-1.09770,0.00995,-1.75499,0.576573,18.884943,-17.538258,8,16,0.406702,0.082409,0.06,167,2
5310815,1.0326,1.2565,-0.36445,-0.46785,1.27562,0.450257,-7.451253,11.453720,8,16,0.413586,0.090786,0.06,167,2
5310816,1.0300,1.2162,0.28746,-0.90923,2.11684,0.376957,22.510724,-16.267590,8,16,0.420453,0.098894,0.06,167,2
5310817,1.0543,1.1544,0.43512,-1.10460,2.90989,0.172448,0.985146,-3.719230,8,16,0.427482,0.106590,0.06,167,2


In [32]:
df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [33]:
df_train.head()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
0,0.00000,0.00000,0.000000,0.000000,0.0000,0.000000,0.00000,0.00000,0,0,0.000000,0.000000,0.00,0,0
1,-0.00005,0.02900,0.000000,0.959571,0.0000,0.000000,0.00000,23.46909,4,32,-0.124001,0.389720,0.06,0,0
2,0.00000,0.10938,0.000000,1.277536,0.0000,0.000000,0.00000,-3.18073,4,32,-0.124001,0.391178,0.06,0,0
3,0.00000,0.17944,0.000000,1.114890,0.0000,0.000000,0.00000,-2.49506,4,32,-0.124001,0.393571,0.06,0,0
4,-0.03796,0.22632,-1.078362,0.612950,-1.4809,-0.712811,-14.55815,-3.50009,4,32,-0.124507,0.396589,0.06,0,0


In [39]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,16000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,16000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,16000)


Length: 2655282 races: 168 n batches: 2641842 / 512
Length: 1327640 races: 168 n batches: 1314200 / 512
Length: 1327896 races: 168 n batches: 1314456 / 512


In [35]:
x_y=gen_test[0]
x=x_y[0][0]
print(x)
x_y[1][0]

tf.Tensor(
[[ 7.2567000e-01 -1.3595000e-01 -1.7827000e+00  1.7000000e+01
   2.2000000e+01 -3.0831870e-01  4.9901690e-01]
 [ 3.3367000e-01 -3.2162000e-01 -1.1488000e+00  1.7000000e+01
   2.2000000e+01 -3.2057290e-01  4.7317060e-01]
 [ 3.0362000e-01 -3.0650000e-01 -5.2820000e-01  1.7000000e+01
   2.2000000e+01 -3.3252930e-01  4.4704100e-01]
 [ 2.9529000e-01 -3.1972000e-01 -2.8810000e-01  1.7000000e+01
   2.2000000e+01 -3.4426110e-01  4.2068030e-01]
 [ 2.3114000e-01 -1.1164000e-01 -1.7960000e-01  1.7000000e+01
   2.2000000e+01 -3.5579920e-01  3.9414060e-01]
 [ 1.0494000e-01 -2.0519000e-01 -4.0200000e-02  1.7000000e+01
   2.2000000e+01 -3.6721030e-01  3.6748050e-01]
 [ 6.1900000e-02 -7.7540000e-02 -1.6100000e-02  1.7000000e+01
   2.2000000e+01 -3.7856280e-01  3.4072920e-01]
 [-9.5300000e-03 -1.5519000e-01 -2.2000000e-03  1.7000000e+01
   2.2000000e+01 -3.8990560e-01  3.1387700e-01]
 [ 1.2020000e-02 -5.5090000e-02 -5.9000000e-03  1.7000000e+01
   2.2000000e+01 -4.0123530e-01  2.8697920e-01]

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 0.1117319, -0.27888  , -0.2056   ])>

### Model definition

#### Train and print

In [40]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [41]:
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=10,verbose=1,name="MLP_DELTA",path="logs")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/MLP_DELTA')

Epoch 1/10
5158/5158 [==============================] - 521s 101ms/step - loss: 3.3003 - mean_absolute_error: 0.8741 - accuracy: 1.2622e-07 - val_loss: 0.4709 - val_mean_absolute_error: 0.4252 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/10
5158/5158 [==============================] - 524s 102ms/step - loss: 0.3405 - mean_absolute_error: 0.3410 - accuracy: 2.5244e-07 - val_loss: 0.2899 - val_mean_absolute_error: 0.3091 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5158/5158 [==============================] - 523s 101ms/step - loss: 0.2827 - mean_absolute_error: 0.2969 - accuracy: 0.0000e+00 - val_loss: 0.2763 - val_mean_absolute_error: 0.2979 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5158/5158 [==============================] - 522s 101ms/step - loss: 0.2783 - mean_absolute_error: 0.2918 - accuracy: 1.2622e-07 - val_loss: 0.2751 - val_mean_absolute_error: 0.2968 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
5157/5158 [============================>.

In [ ]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

In [42]:
lr =1e-4
units=8
cells=1
normalization=False
name=f"LSTM{cells}_{units}_{lr}_ROT_DELTA"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=6,name=name,path="logs",
                                                     epoch=100, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/{name}')

Epoch 1/100
5158/5158 [==============================] - 521s 101ms/step - loss: 0.8483 - mean_absolute_error: 0.6047 - accuracy: 0.0000e+00 - val_loss: 0.5499 - val_mean_absolute_error: 0.4992 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/100
5158/5158 [==============================] - 517s 100ms/step - loss: 0.5481 - mean_absolute_error: 0.4767 - accuracy: 1.2622e-07 - val_loss: 0.3774 - val_mean_absolute_error: 0.3940 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/100
5158/5158 [==============================] - 518s 100ms/step - loss: 0.4579 - mean_absolute_error: 0.4230 - accuracy: 1.2622e-07 - val_loss: 0.3484 - val_mean_absolute_error: 0.3692 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/100
5158/5158 [==============================] - 517s 100ms/step - loss: 0.4204 - mean_absolute_error: 0.4021 - accuracy: 1.2622e-07 - val_loss: 0.3345 - val_mean_absolute_error: 0.3610 - val_accuracy: 2.5382e-07 - lr: 1.0000e-04
Epoch 5/100
5158/5158 [=========================

In [27]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}_PosAssoluta')

INFO:tensorflow:Assets written to: saved_model/LSTM2_32_1e-05_NormFalse_PosAssoluta\assets


INFO:tensorflow:Assets written to: saved_model/LSTM2_32_1e-05_NormFalse_PosAssoluta\assets


In [19]:
model=model_definitions.CNN_LSTM_Model(0.2, units=32,filters=32,kernel_size=1, normalization=False)
model.load_weights("saved_model\CNN_LSTM32_1_32_PosAssoluta")

In [20]:
model.summary()

Model: "CNN32_1LSTM32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 30, 5)]           0         
                                                                 
 conv1d (Conv1D)             (None, 30, 32)            192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 32)           0         
 )                                                               
                                                                 
 lstm_30 (LSTM)              (None, 32)                8320      
                                                                 
 dense_6 (Dense)             (None, 3)                 99        
                                                                 
Total params: 8,611
Trainable params: 8,611
Non-trainable params: 0
___________________________________________________

In [ ]:
import tf2onnx
import onnx
input_signature=(tf.TensorSpec([None,30,5],tf.float32,name='x'))
onnx_model,_=tf2onnx.converter.from_keras(model,input_signature,opset=9)